In [1]:
#Import Libraries
import cv2
import itertools
import numpy as np
from time import time
import mediapipe as mp
import matplotlib.pyplot as plt
import pandas as pd
import re
import glob
import os
import pickle
import json

from tqdm import tqdm
from skimage import transform
import concurrent.futures

In [2]:
# Initialize the mediapipe face detection class
mp_face_detection = mp.solutions.face_detection
# Setup the face detection function
face_detection = mp_face_detection.FaceDetection(model_selection=0, min_detection_confidence=0.5)
#initialize the mediapipe face mesh class
mp_face_mesh = mp.solutions.face_mesh
# Setup the face landmarks function for videos
face_mesh_videos = mp_face_mesh.FaceMesh(static_image_mode=False, max_num_faces=1,
                                         min_detection_confidence=0.5,min_tracking_confidence=0.3)

In [3]:
def save2npz(filename, data=None):
    if not os.path.exists(os.path.dirname(filename)):
        os.makedirs(os.path.dirname(filename))
    np.savez_compressed(filename, data=data)

In [4]:
def extract_points_from_mesh(face_landmarks, indexes):
    points_data_regex = re.compile(r'\d\.\d+')
    xy_points_list = []
    for count, each_index in enumerate(indexes):
        xyzpointsraw = face_landmarks.landmark[each_index]
        points_list = points_data_regex.findall(str(xyzpointsraw))
        if len(points_list) < 1:
            xy_points_list.append([None])
        else:
            xyclean = [float(points_list[0]), float(points_list[1])]
            xy_points_list.append(xyclean)
    xy_points_array = np.array(xy_points_list)
    return xy_points_array

In [5]:
def detect_Facial_Landmarks(image):
    frame_dict = {}
    mesh_result = face_mesh_videos.process(image)
    oval_indexes = list(set(itertools.chain(*mp_face_mesh.FACEMESH_FACE_OVAL)))
    lips_indexes = list(set(itertools.chain(*mp_face_mesh.FACEMESH_LIPS)))
    if mesh_result.multi_face_landmarks:
        for face_no, face_landmarks in enumerate(mesh_result.multi_face_landmarks):
            oval_points_array = extract_points_from_mesh(face_landmarks, oval_indexes)
            lips_points_array = extract_points_from_mesh(face_landmarks, lips_indexes)
            frame_dict['oval_landmarks'] = oval_points_array
            frame_dict['lips_landmarks'] = lips_points_array
    else:
        frame_dict['oval_landmarks'] = None
        frame_dict['lips_landmarks'] = None
    return frame_dict

In [6]:
def get_face_points(video):
    vid_capture = cv2.VideoCapture(video)
    all_points = []
    if (vid_capture.isOpened() == False):
        print("Error opening the video file")
    else:
        while(vid_capture.isOpened()):
            ret, frame = vid_capture.read()
            if ret:
                frame_points_dict = detect_Facial_Landmarks(frame)
                all_points.append(frame_points_dict)
                k = cv2.waitKey(30)
                if(k == 27):
                    break
            else:
                break
    vid_capture.release()
    cv2.destroyAllWindows()
    return np.array(all_points)

In [7]:
def landmarks_extract_save(video, landmarks_output):
    dir, file = os.path.split(video)
    part, which_folder = os.path.split(dir)
    otherpart, word = os.path.split(part)
    new_filename = file[:-4] + ".npz"
    new_save_output = os.path.join(landmarks_output, word, which_folder, new_filename)
    if not os.path.exists(new_save_output):
        face_points_array = get_face_points(video)
        save2npz(new_save_output, face_points_array)

In [8]:
#Has issues with mediapipe and multiprocess
def multi_extract_landmarks(lrw_direc, landmarks_output):
    videos = glob.glob(os.path.join(lrw_direc, '*', which_folder, '*.mp4'))
    l= len(videos)
    with tqdm(total = l) as pbar:
        with concurrent.futures.ThreadPoolExecutor(max_workers=4) as executor:
            futures = [executor.submit(landmarks_extract_save, video, landmarks_output) for video in videos]
            for future in concurrent.futures.as_completed(futures):
                pbar.update(1)

In [9]:
lrw_path = '/media/taylorpap/1TBM2/DatasetML/lipread_mp4'
landmarks_path = '/home/taylorpap/Bootcamp/LANDMARKS'
#which_folder = 'test'
which_folder = 'train'

In [1]:
#multi_extract_landmarks(lrw_path, landmarks_path)

In [16]:
videos = glob.glob(os.path.join(lrw_path, '*', which_folder, '*.mp4'))
for video in tqdm(videos):
    landmarks_extract_save(video, landmarks_path)

100%|██████████| 25000/25000 [1:07:44<00:00,  6.15it/s]


In [17]:
which_folder = 'val'
videos = glob.glob(os.path.join(lrw_path, '*', which_folder, '*.mp4'))
for video in tqdm(videos):
    landmarks_extract_save(video, landmarks_path)

100%|██████████| 25000/25000 [1:08:31<00:00,  6.08it/s]


In [10]:
temp_words_list = ['ABSOLUTELY', 'BUDGET', 'EVERYONE', 'HOUSE', 'MILITARY', 'PUBLIC', 'RESULT', 'SIGNIFICANT', 'WEATHER']
for temp_word in temp_words_list:
    videos = glob.glob(os.path.join(lrw_path, temp_word, which_folder, '*.mp4'))
    for video in tqdm(videos):
        landmarks_extract_save(video, landmarks_path)

100%|██████████| 1000/1000 [02:54<00:00,  5.72it/s]
